In [27]:
%pip install tabulate
import math
import random

Note: you may need to restart the kernel to use updated packages.


In [28]:
# Variável que habilita o print das iterações (Tabelas, médias, etc)
printON = False

In [29]:
# Função para gerar números pseudo-aleatórios sem filtro
def gerar_pseudo_aleatorio(semente, n):
    resultados = []
    modulo = (2 ** 31) - 1
    multiplicador = 16807
    c = 1
    x = semente

    for _ in range(n):
        x = ((multiplicador * x) + c) % modulo
        resultados.append(x / modulo)

    return resultados

In [30]:
# Função para determinar o tipo do cliente com base no número pseudo-aleatório
def determinar_tipo(cliente_aleatorio):
    if 0 <= cliente_aleatorio < 0.6:
        return 1
    elif 0.6 <= cliente_aleatorio < 0.9:
        return 2
    else:
        return 3

In [31]:
# Funções de cálculo

def calcular_tempo_medio_espera_fila(clientes):
    soma_fila = sum(cliente['tempo_fila'] for cliente in clientes)
    media_fila = soma_fila / len(clientes)

    if printON:
        print(f"\nTempo Médio de Espera na Fila: {media_fila:.2f}")
    return media_fila

def calcular_media_ts(clientes):
    soma_ts = sum(cliente['ts'] for cliente in clientes)
    media_ts = soma_ts / len(clientes)

    if printON:
        print(f"\nTempo Médio de Serviço (TS): {media_ts:.2f}")
    return media_ts

def calcular_media_tempo_sistema(atendidos):
    soma_sistema = sum(cliente['tempo_sistema'] for cliente in atendidos)
    media_sistema = soma_sistema / len(atendidos)

    if printON:
        print(f"\nTempo Médio no Sistema: {media_sistema:.2f}")
    return media_sistema

def calcular_media_ociosidade(funcionarios):
    total_ociosidade = sum(funcionario['ocio'] for funcionario in funcionarios)
    total_atendimentos = sum(funcionario['num_atendimentos'] for funcionario in funcionarios)

    media_ociosidade_total = total_ociosidade / total_atendimentos if total_atendimentos > 0 else 0

    if printON:
        print(f"\nTempo Médio Ocioso Total: {media_ociosidade_total:.2f}")
    return media_ociosidade_total

def calcular_desvio_padrao(numeros, media):
    soma_quadrados = sum((x - media) ** 2 for x in numeros)
    return math.sqrt(soma_quadrados / (len(numeros) - 1))

def calcular_intervalo_confianca(medias, z=1.96):
    n = len(medias)
    media = sum(medias) / n
    desvio_padrao = calcular_desvio_padrao(medias, media)
    margem_erro = z * (desvio_padrao / math.sqrt(n))

    return {
        'inferior': media - margem_erro,
        'superior': media + margem_erro
    }

def calcular_tec(aleatorio):
    return -15 * math.log(aleatorio)

def calcular_ts(tipo, aleatorio):
    if tipo == 1:
        return -15 * math.log(aleatorio) + 15
    elif tipo == 2:
        return -40 * math.log(aleatorio) + 30
    else:
        return -140 * math.log(aleatorio) + 60

In [32]:
# Função de atendimento do cliente
def atender_cliente(funcionario, cliente):
    cliente['inicio_servico'] = max(cliente['chegada'], funcionario['fim_atendimento'])
    cliente['fim_servico'] = cliente['inicio_servico'] + cliente['ts']
    cliente['tempo_fila'] = cliente['inicio_servico'] - cliente['chegada']
    cliente['tempo_sistema'] = cliente['ts'] + cliente['tempo_fila']
    cliente['funcionario_id'] = funcionario['id']
    cliente['tempo_ocioso_funcionario'] = cliente['inicio_servico'] - funcionario['fim_atendimento']

    funcionario['ocio'] += cliente['tempo_ocioso_funcionario']
    funcionario['num_atendimentos'] += 1
    funcionario['disponivel'] = False
    funcionario['fim_atendimento'] = cliente['fim_servico']
    funcionario['atendendo'] = cliente

In [33]:
# Função para simular o banco
def simular_banco(semente, num_clientes, num_funcionarios, tempo_maximo):
    funcionarios = [{'id': i + 1, 'ocio': 0, 'disponivel': True, 'fim_atendimento': 0, 'num_atendimentos': 0} for i in range(num_funcionarios)]

    tamanho_total = num_clientes * 3
    numeros_aleatorios = gerar_pseudo_aleatorio(semente, tamanho_total)

    numeros_aleatorios_tipo = numeros_aleatorios[:num_clientes]
    numeros_aleatorios_tec = numeros_aleatorios[num_clientes:num_clientes * 2]
    numeros_aleatorios_ts = numeros_aleatorios[num_clientes * 2:]

    clientes = [
        {
            'id': i + 1,
            'tipo': determinar_tipo(numeros_aleatorios_tipo[i]),
            'chegada': 0,
            'inicio_servico': 0,
            'fim_servico': 0,
            'tempo_fila': 0,
            'tempo_sistema': 0,
            'tec': calcular_tec(numeros_aleatorios_tec[i]),
            'ts': calcular_ts(determinar_tipo(numeros_aleatorios_tipo[i]), numeros_aleatorios_ts[i])
        }
        for i in range(num_clientes)
    ]

    fila = []
    tempo_simulacao = 0
    atendidos = []
    i = 0

    while i < num_clientes or fila or any(not func['disponivel'] for func in funcionarios):
        cliente = clientes[i] if i < num_clientes else None

        fim_atendimento_disponiveis = [func['fim_atendimento'] for func in funcionarios if not func['disponivel']]
        tempo_simulacao = min(
            cliente['chegada'] if cliente else float('inf'),
            min(fim_atendimento_disponiveis) if fim_atendimento_disponiveis else float('inf')
        )

        if cliente and cliente['chegada'] <= tempo_maximo:
            if i == 0:
                cliente['chegada'] = cliente['tec']
            else:
                cliente['chegada'] = clientes[i - 1]['chegada'] + cliente['tec']

            funcionario_disponivel = next((func for func in funcionarios if func['disponivel'] and func['fim_atendimento'] <= cliente['chegada']), None)

            if funcionario_disponivel:
                atender_cliente(funcionario_disponivel, cliente)
                atendidos.append(cliente)
                i += 1
            else:
                if cliente['chegada'] <= tempo_maximo:
                    fila.append(cliente)
                i += 1

        for funcionario in funcionarios:
            if not funcionario['disponivel'] and funcionario['fim_atendimento'] <= tempo_simulacao:
                funcionario['disponivel'] = True
                if fila:
                    proximo_cliente = fila.pop(0)
                    atender_cliente(funcionario, proximo_cliente)
                    atendidos.append(proximo_cliente)

    media_fila = calcular_tempo_medio_espera_fila(clientes)
    media_ts = calcular_media_ts(atendidos)
    media_sistema = calcular_media_tempo_sistema(atendidos)
    media_ociosidade = calcular_media_ociosidade(funcionarios)

    return {
        'media_fila': media_fila,
        'media_ts': media_ts,
        'media_sistema': media_sistema,
        'media_ociosidade': media_ociosidade
    }

In [34]:
from tabulate import tabulate

# Simulação principal
medias = []

# Para várias sementes
for semente in range(1, 1000):
    resultado = simular_banco(semente, 120, 4, 1800)
    medias.append(resultado)

arr_filas = [resultado['media_fila'] for resultado in medias]
arr_ts = [resultado['media_ts'] for resultado in medias]
arr_sistema = [resultado['media_sistema'] for resultado in medias]
arr_ociosidade = [resultado['media_ociosidade'] for resultado in medias]

intervalo_fila = calcular_intervalo_confianca(arr_filas)
intervalo_ts = calcular_intervalo_confianca(arr_ts)
intervalo_sistema = calcular_intervalo_confianca(arr_sistema)
intervalo_ociosidade = calcular_intervalo_confianca(arr_ociosidade)

# Preparando os dados para a tabela
dados = [
    ["Tempo na Fila", f"[{intervalo_fila['inferior']:.2f}, {intervalo_fila['superior']:.2f}]"],
    ["Tempo de Serviço", f"[{intervalo_ts['inferior']:.2f}, {intervalo_ts['superior']:.2f}]"],
    ["Tempo no Sistema", f"[{intervalo_sistema['inferior']:.2f}, {intervalo_sistema['superior']:.2f}]"],
    ["Tempo em Ociosidade", f"[{intervalo_ociosidade['inferior']:.2f}, {intervalo_ociosidade['superior']:.2f}]"]
]

# Exibindo os dados em formato de tabela
print(tabulate(dados, headers=["Métrica", "Intervalo de Confiança"], tablefmt="grid", numalign="center"))

+---------------------+------------------------+
|       Métrica       | Intervalo de Confiança |
+---------------------+------------------------+
|    Tempo na Fila    |     [74.37, 82.94]     |
|   Tempo de Serviço  |     [58.31, 59.13]     |
|   Tempo no Sistema  |    [134.62, 143.77]    |
| Tempo em Ociosidade |      [8.09, 8.77]      |
+---------------------+------------------------+
